In [ ]:
#please note the AWS architecture was not created for this assessment
#the routes have to be updated when running this script

# importing libraries for the script
import json
import boto3
from PIL import Image

#defining lambda function triggered by S3 object creation events
def lambda_handler(event, context):
# Getting the S3 object information from the event
  try:
    s3_record = event['Records'][0]['s3']
    bucket_name = s3_record['bucket']['name']
    object_key = s3_record['object']['key']

# Setting up S3 client
    s3_client = boto3.client('s3')

# Downloading the image from S3
    s3_client.download_file(bucket_name, object_key, "/tmp/input_image.jpg")

# Generating the thumbnail
    im = Image.open("/tmp/input_image.jpg")
    thumbnail_size = (128, 128)  # Adjust thumbnail size as needed
    im.thumbnail(thumbnail_size, Image.ANTIALIAS)

# Saving the thumbnail to a temporary file
    im.save("/tmp/thumbnail.jpg")

# Uploading the thumbnail to a destination S3 bucket
    thumbnail_bucket_name = "thumbnail-bucket"
    thumbnail_object_key = f"thumbnails/{filename}"
    s3_client.upload_file("/tmp/thumbnail.jpg", thumbnail_bucket_name, thumbnail_object_key)

    print(f"Thumbnail generated and uploaded: s3://{thumbnail_bucket_name}/{thumbnail_object_key}")
    return {
      'statusCode': 200,
      'body': json.dumps('Thumbnail generation successful!')
    }

  except Exception as e:
    print(f"Error generating thumbnail: {e}")
    return {
      'statusCode': 500,
      'body': json.dumps(f"Error processing image: {e}")
    }
# Cleaning up temporary files
  finally:
    try:
      os.remove("/tmp/input_image.jpg")
      os.remove("/tmp/thumbnail.jpg")
    except FileNotFoundError:
      pass

## Example usage within Terraform local-exec
#s3_client = boto3.client('s3')  # Assume appropriate credentials are configured
# ... Other Terraform configuration
#resource "null_resource" "generate_thumbnail" {
 #provisioner "local-exec" {
    #command = <<EOF
      #python generate_thumbnail.py \
        #--input-bucket-name my-input-bucket \
        #--input-object-key path/to/image.jpg \
        #--thumbnail-bucket-name my-thumbnail-bucket \
        #--thumbnail-object-key thumbnails/resized_image.jpg
    #EOF
  #}
#}